# 🧠 Bias Audit — Complete Analysis Notebook

**What this notebook does (one-run):**
- Mounts Google Drive
- Ensures a working folder exists at `/content/drive/MyDrive/Colab_Models`
- Creates a sample `bias_audit.xlsx` if none exists
- Loads dataset
- Data quality checks
- Computes group distributions for sensitive features
- Visualizes sorted charts for Positive/Negative/Neutral outcomes
- Calculates fairness metrics:
  - Demographic parity ratio (Female vs Male if present)
  - Equalized odds (False Positive Rate & False Negative Rate) if `True_Label` present
- Produces a findings summary (printed)
- Saves charts and a full Excel report to Drive


In [3]:
# 0) Imports
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
from datetime import datetime

sns.set(style="whitegrid", rc={"figure.dpi": 120})


In [4]:
# 1) Mount Google Drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# 2) Setup folder & sample dataset creation if missing
FOLDER_PATH = '/content/drive/MyDrive/Colab_Models'
os.makedirs(FOLDER_PATH, exist_ok=True)
DATA_FILENAME = 'bias_audit.xlsx'
FULL_PATH = os.path.join(FOLDER_PATH, DATA_FILENAME)

# If user doesn't have a dataset, create a realistic sample dataset
if not os.path.exists(FULL_PATH):
    print("No dataset found. Creating a sample bias_audit.xlsx in your Drive...")
    rng = np.random.default_rng(42)
    n = 200
    genders = rng.choice(['Male','Female','Non-binary'], size=n, p=[0.45,0.45,0.10])
    ages = rng.integers(18,70,size=n)

    # Simulate predictions with slight bias
    logits = (ages - ages.mean())*0.02 + (genders == 'Male')*0.1 + rng.normal(0, 0.5, size=n)
    probs = 1 / (1 + np.exp(-logits))
    predictions = np.where(probs > 0.55, 'Positive', np.where(probs < 0.45, 'Negative', 'Neutral'))

    # True labels for equalized odds
    true_logits = (ages - ages.mean())*0.01 + rng.normal(0,0.6,size=n)
    true_probs = 1 / (1 + np.exp(-true_logits))
    true_labels = np.where(true_probs > 0.5, 'Positive', 'Negative')

    df_sample = pd.DataFrame({
        'ID': np.arange(1, n+1),
        'Gender': genders,
        'Age': ages,
        'Prediction': predictions,
        'True_Label': true_labels
    })
    df_sample.to_excel(FULL_PATH, index=False)
    print(f"Sample dataset created: {FULL_PATH}")


In [6]:
# 3) Load dataset
print("\nLoading dataset...")
df = pd.read_excel(FULL_PATH)
print("Dataset preview:")
display(df.head())
print("\nDataset info:")
print(df.info())
print("\nMissing values per column:")
print(df.isna().sum())



Loading dataset...
Dataset preview:


,ID,Gender,Age,Prediction
0,1,Male,25,Positive
1,2,Female,30,Negative
2,3,Female,22,Positive
3,4,Male,45,Neutral
4,5,Male,35,Negative



Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID          8 non-null      int64 
 1   Gender      8 non-null      object
 2   Age         8 non-null      int64 
 3   Prediction  8 non-null      object
dtypes: int64(2), object(2)
memory usage: 388.0+ bytes
None

Missing values per column:
ID            0
Gender        0
Age           0
Prediction    0
dtype: int64


In [7]:
# 4) Settings: define sensitive features & outcome
sensitive_cols = ['Gender', 'Age']  # Age will be bucketed
outcome_col = 'Prediction'
true_label_col = 'True_Label'  # set to None if no true labels

# 5) Preprocess Age into categories
if 'Age' in df.columns and 'Age' in sensitive_cols:
    bins = [0, 25, 35, 50, 200]
    labels = ['18-25', '26-35', '36-50', '51+']
    df['AgeGroup'] = pd.cut(df['Age'], bins=bins, labels=labels, right=True)
    sensitive_cols = ['Gender'] + [c for c in sensitive_cols if c != 'Age'] + ['AgeGroup']

print("\nFinal sensitive features examined:", sensitive_cols)



Final sensitive features examined: ['Gender', 'Gender', 'AgeGroup']


In [8]:
# 6) Helper functions
def group_proportions(df, sensitive_col, outcome_col):
    grouped = df.groupby(sensitive_col)[outcome_col].value_counts(normalize=True).unstack().fillna(0)
    cols = [c for c in ['Positive','Neutral','Negative'] if c in grouped.columns]
    grouped = grouped[cols]
    return grouped

def demographic_parity_ratio(grouped, ref_group=None, target='Positive'):
    if target not in grouped.columns:
        return None, None
    if ref_group is None:
        ref_group = grouped[target].idxmax()
    minority_group = grouped[target].idxmin()
    ratio = grouped.loc[minority_group, target] / max(grouped.loc[ref_group, target], 1e-9)
    return ratio, (ref_group, minority_group)

def equalized_odds(df, sensitive_col, outcome_col, true_label_col):
    results = {}
    if true_label_col not in df.columns:
        return None
    for grp in df[sensitive_col].dropna().unique():
        grp_df = df[df[sensitive_col] == grp]
        FP = ((grp_df[outcome_col] == 'Positive') & (grp_df[true_label_col] == 'Negative')).sum()
        TN = ((grp_df[outcome_col] != 'Positive') & (grp_df[true_label_col] == 'Negative')).sum()
        FN = ((grp_df[outcome_col] != 'Positive') & (grp_df[true_label_col] == 'Positive')).sum()
        TP = ((grp_df[outcome_col] == 'Positive') & (grp_df[true_label_col] == 'Positive')).sum()
        FPR = FP / (FP + TN) if (FP + TN) > 0 else np.nan
        FNR = FN / (FN + TP) if (FN + TP) > 0 else np.nan
        results[grp] = {'False_Positive_Rate': float(FPR) if not np.isnan(FPR) else None,
                        'False_Negative_Rate': float(FNR) if not np.isnan(FNR) else None,
                        'Support': int(len(grp_df))}
    return results


In [9]:
# 7) Analysis, charts, and report
report_rows = []
charts_saved = []
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

for col in sensitive_cols:
    if col not in df.columns:
        print(f"Skipping {col} — column not present.")
        continue
    print(f"\n=== ANALYSIS: {col} ===")
    gp = group_proportions(df, col, outcome_col)
    gp_sorted = gp.sort_values(by='Positive', ascending=False) if 'Positive' in gp.columns else gp
    display(gp_sorted)

    # Demographic parity
    dp_ratio, groups = demographic_parity_ratio(gp_sorted) if 'Positive' in gp_sorted.columns else (None, None)
    if dp_ratio is not None:
        print(f"Demographic parity ratio (min/major): {dp_ratio:.3f} between groups {groups[1]} and {groups[0]}")

    # Equalized odds
    eo = equalized_odds(df, col, outcome_col, true_label_col) if true_label_col in df.columns else None
    if eo:
        print("Equalized odds per group:")
        for k,v in eo.items():
            print(f"  {k}: FPR={v['False_Positive_Rate']}, FNR={v['False_Negative_Rate']}, N={v['Support']}")

    # Visualizations
    plot_df = gp_sorted.reset_index().melt(id_vars=col, var_name='Outcome', value_name='Proportion')
    plt.figure(figsize=(8,4))
    sns.barplot(data=plot_df[plot_df['Outcome']=='Positive'], x=col, y='Proportion')
    plt.title(f'Positive Prediction Proportion by {col}')
    plt.xticks(rotation=45)
    chart_path_pos = os.path.join(FOLDER_PATH, f'{timestamp}_{col}_positive.png')
    plt.tight_layout()
    plt.savefig(chart_path_pos)
    plt.close()
    charts_saved.append(chart_path_pos)

    fig, ax = plt.subplots(figsize=(9,5))
    gp_sorted.plot(kind='bar', stacked=True, ax=ax)
    ax.set_title(f'Prediction distribution by {col}')
    ax.set_ylabel('Proportion')
    plt.xticks(rotation=45)
    stacked_chart_path = os.path.join(FOLDER_PATH, f'{timestamp}_{col}_stacked.png')
    plt.tight_layout()
    fig.savefig(stacked_chart_path)
    plt.close(fig)
    charts_saved.append(stacked_chart_path)

    # Add rows to report
    for grp in gp_sorted.index:
        row = {
            'Sensitive_Feature': col,
            'Group': grp,
            'Positive_Proportion': float(gp_sorted.loc[grp].get('Positive', 0)),
            'Neutral_Proportion': float(gp_sorted.loc[grp].get('Neutral', 0)),
            'Negative_Proportion': float(gp_sorted.loc[grp].get('Negative', 0)),
            'Demographic_Parity_Ratio': float(dp_ratio) if dp_ratio is not None else None,
            'Stacked_Chart': stacked_chart_path,
            'Positive_Chart': chart_path_pos,
            'Equalized_Odds': eo.get(grp) if eo else None
        }
        report_rows.append(row)

print("\nCharts saved to Drive:")
for c in charts_saved:
    print("  -", c)

# Save full report
report_df = pd.DataFrame(report_rows)
report_file = os.path.join(FOLDER_PATH, f'bias_audit_report_{timestamp}.xlsx')
report_df.to_excel(report_file, index=False)
print(f"\n✅ Full report saved to: {report_file}")



=== ANALYSIS: Gender ===


Prediction,Positive,Neutral,Negative
Gender,,,
Female,0.5,0.00,0.50
Male,0.5,0.25,0.25


Demographic parity ratio (min/major): 1.000 between groups Female and Female

=== ANALYSIS: Gender ===


Prediction,Positive,Neutral,Negative
Gender,,,
Female,0.5,0.00,0.50
Male,0.5,0.25,0.25


Demographic parity ratio (min/major): 1.000 between groups Female and Female

=== ANALYSIS: AgeGroup ===


/tmp/ipython-input-524592357.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby(sensitive_col)[outcome_col].value_counts(normalize=True).unstack().fillna(0)


Prediction,Positive,Neutral,Negative
AgeGroup,,,
18-25,1.000000,0.000000,0.000000
26-35,0.333333,0.000000,0.666667
36-50,0.333333,0.333333,0.333333
51+,0.000000,0.000000,0.000000


Demographic parity ratio (min/major): 0.000 between groups 51+ and 18-25

Charts saved to Drive:
  - /content/drive/MyDrive/Colab_Models/20251008_102005_Gender_positive.png
  - /content/drive/MyDrive/Colab_Models/20251008_102005_Gender_stacked.png
  - /content/drive/MyDrive/Colab_Models/20251008_102005_Gender_positive.png
  - /content/drive/MyDrive/Colab_Models/20251008_102005_Gender_stacked.png
  - /content/drive/MyDrive/Colab_Models/20251008_102005_AgeGroup_positive.png
  - /content/drive/MyDrive/Colab_Models/20251008_102005_AgeGroup_stacked.png

✅ Full report saved to: /content/drive/MyDrive/Colab_Models/bias_audit_report_20251008_102005.xlsx


In [10]:
# 9) Findings summary
summary_lines = []
for col in sensitive_cols:
    if col not in df.columns and col != 'AgeGroup':
        continue
    gp = group_proportions(df, col, outcome_col)
    if 'Positive' not in gp.columns:
        continue
    gp_sorted = gp.sort_values(by='Positive', ascending=False)
    best = gp_sorted['Positive'].idxmax()
    worst = gp_sorted['Positive'].idxmin()
    summary_lines.append(f"For **{col}**, group **{best}** has the highest Positive rate ({gp_sorted['Positive'].max():.2f}) "
                         f"and group **{worst}** has the lowest ({gp_sorted['Positive'].min():.2f}).")

    dp_ratio, groups = demographic_parity_ratio(gp_sorted)
    if dp_ratio is not None:
        summary_lines.append(f"  - Demographic parity ratio (min/major) = {dp_ratio:.2f} (min={groups[1]}, major={groups[0]}).")

if true_label_col in df.columns:
    eo_all = {}
    for col in sensitive_cols:
        eo = equalized_odds(df, col, outcome_col, true_label_col)
        if eo:
            eo_all[col] = eo
    if eo_all:
        summary_lines.append("Equalized odds (FPR/FNR) computed per group. Review the report for numeric values and groups with highest disparity.")

for line in summary_lines:
    print(line)


For **Gender**, group **Female** has the highest Positive rate (0.50) and group **Female** has the lowest (0.50).
  - Demographic parity ratio (min/major) = 1.00 (min=Female, major=Female).
For **Gender**, group **Female** has the highest Positive rate (0.50) and group **Female** has the lowest (0.50).
  - Demographic parity ratio (min/major) = 1.00 (min=Female, major=Female).
For **AgeGroup**, group **18-25** has the highest Positive rate (1.00) and group **51+** has the lowest (0.00).
  - Demographic parity ratio (min/major) = 0.00 (min=51+, major=18-25).


/tmp/ipython-input-524592357.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby(sensitive_col)[outcome_col].value_counts(normalize=True).unstack().fillna(0)


In [12]:
!pip install fpdf pillow


  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=b09cd9d25cea1e93d24990fc196ed2536f0e8d489b7544a61ee099516d035c03
  Stored in directory: /root/.cache/pip/wheels/6e/62/11/dc73d78e40a218ad52e7451f30166e94491be013a7850b5d75
Successfully built fpdf


In [13]:
from fpdf import FPDF
from PIL import Image
# rest of your PDF code...


In [14]:
from fpdf import FPDF
from PIL import Image

pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()
pdf.set_font("Arial", 'B', 16)
pdf.cell(0, 10, "AI Bias Audit Report", ln=True, align="C")
pdf.ln(10)

pdf.set_font("Arial", '', 12)
for line in summary_lines:
    pdf.multi_cell(0, 8, line)

pdf.ln(5)
pdf.set_font("Arial", 'B', 14)
pdf.cell(0, 8, "Charts", ln=True)
pdf.ln(5)

for chart in charts_saved:
    pdf.add_page()
    pdf.image(chart, w=180)  # Adjust width as needed

pdf_file = os.path.join(FOLDER_PATH, f'bias_audit_report_{timestamp}.pdf')
pdf.output(pdf_file)
print(f"\n✅ PDF report saved to: {pdf_file}")



✅ PDF report saved to: /content/drive/MyDrive/Colab_Models/bias_audit_report_20251008_102005.pdf
